In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))



# Any results you write to the current directory are saved as output.

In [ ]:
import os
import pandas as pd  
from matplotlib import pyplot as plt 
import cv2
from sklearn.metrics import roc_auc_score, classification_report 
from pathlib import Path
%matplotlib inline 

In [ ]:
import fastai
from fastai.vision import *

In [ ]:
work_dir = Path('/kaggle/working/')
path = Path('../input') 


train = path/'train_images/train_images'
test =  path/'leaderboard_test_data/leaderboard_test_data'
holdout = path/'leaderboard_holdout_data/leaderboard_holdout_data/'
sample_sub = path/'SampleSubmission.csv'
labels = path/'traininglabels.csv'

df = pd.read_csv(labels)
df_sample = pd.read_csv(sample_sub) 


In [ ]:
df.head(2)

In [ ]:
df.has_oilpalm.hist()  #imbalanced  

In [ ]:
test_names = [f for f in test.iterdir()]
holdout_names = [f for f in holdout.iterdir()]

In [ ]:
#read imgs: 
src = (ImageItemList.from_df(df, path, folder=train)
      .random_split_by_pct(0.2, seed=2019)
      .label_from_df('has_oilpalm')
      .add_test(test_names+holdout_names)) 


In [ ]:
data =  (src.transform(get_transforms(), size=128)  #256 isn't working  
         .databunch(bs=64)
         .normalize(imagenet_stats))

In [ ]:
#data.show_batch(2, figsize=(10,7)) 

In [ ]:
#!wget https://download.pytorch.org/models/vgg19-dcbb9e9d.pth 

In [ ]:
import torch 
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset 

learn = create_cnn(data, models.resnet18, 
                   metrics=[accuracy],  
                   model_dir='/kaggle/working/models')

In [ ]:
lr = 1e-2 

In [ ]:
learn.fit_one_cycle(3, lr)  
p,t = learn.get_preds()
print (classification_report(list(t), list(np.argmax(p, 1) )))

In [ ]:
# Vgg16_bn  

In [ ]:
learn2 = create_cnn(data, models.vgg16_bn, 
                   metrics=[accuracy],  
                   model_dir='/kaggle/working/models')

In [ ]:
learn2.fit_one_cycle(3, lr)  

In [ ]:
p,t = learn2.get_preds()
print (classification_report(list(t), list(np.argmax(p, 1) ))) 

In [ ]:
from sklearn.metrics import auc

In [ ]:
## vgg19_bn 
learn3 = create_cnn(data, models.vgg19_bn, 
                   metrics=[accuracy],  
                   model_dir='/kaggle/working/models')

In [ ]:
learn3.fit_one_cycle(3, lr) 
p,t = learn3.get_preds()
print (classification_report(list(t), list(np.argmax(p, 1) ))) 

In [ ]:
#resnet152
learn4 = create_cnn(data, models.resnet152, 
                   metrics=[accuracy],  
                   model_dir='/kaggle/working/models')

In [ ]:
learn4.fit_one_cycle(3, lr) 
p,t = learn4.get_preds()
print (classification_report(list(t), list(np.argmax(p, 1) ))) 

In [ ]:
df.head(3)

In [ ]:
## keras  
from PIL import Image
import cv2
from subprocess import check_output 
from sklearn.model_selection  import train_test_split 

import csv
import time

train='../input/train_images/train_images'

def img_as_array(image_id, new_img_size):
    imagepath = train + '/' + str(image_id) 
    img = cv2.imread(imagepath)
    resized_image = cv2.resize(img, new_img_size)
    #avg = averages(resized_image,new_img_size)
    
    return resized_image 

def averages(arr,img_size):
    img = np.reshape(arr,(img_size[0],img_size[1],3))
    uAvg = np.average(img,axis=0).tolist()
    vAvg = np.average(img,axis=1).tolist()
    wAvg = np.average(img,axis=2).tolist()
    
    out = []
    for x in uAvg+vAvg+wAvg:
        out += x
    return np.array(out)
        

img_size  = (64,64)
imgs_per_loc = 1 
arr_size = 3*img_size[0]*img_size[1] 

X = []
y = []

print('\tReading data...')
for row in df.itertuples():
    image_id2 = row[1]
    y.append(row[2])
    #print (image_id2)
    img2= img_as_array(image_id2, img_size) 
    X.append(img2)  
    
X = np.array(X)
y = np.array(y) 


X_train, X_test, y_train, y_test= train_test_split(X, y, stratify=y)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(np.array(X_train.reshape(11433, 12288)), np.array(y_train)) 

In [ ]:
X_train_res.shape, y_train_res.shape

In [ ]:
X_train_res=X_train_res.reshape(21452, 64, 64, 3)

In [ ]:
X_train.shape, X_train_res.shape, X_test.shape,  y_train.shape, y_train_res.shape, y_test.shape

In [ ]:
np.bincount(y_train), np.bincount(y_train_res) , np.bincount(y_test) 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

img_width, img_height = 64, 64 

model = Sequential()


model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 1), dim_ordering="tf"))

model.add(Convolution2D(64, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 1), dim_ordering="tf"))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('sigmoid'))


bsize = 32

model.compile(loss='binary_crossentropy',
              optimizer='sgd', #'rmsprop' 
              metrics=['accuracy'])

#class_weight ={0: 1, 1:10}


#history = model.fit_generator(foo,steps_per_epoch=len((X_train)) / bsize, validation_data=(np.array(X_test),np.array(y_test)),verbose=1,epochs=40)
history = model.fit(np.array(X_train_res), np.array(y_train_res),validation_data=(np.array(X_test),np.array(y_test)) ,batch_size=bsize,epochs=10)

In [ ]:
y_pred= model.predict_classes(X_test)
y_pred=y_pred.reshape(-1) 

In [ ]:
print (classification_report(y_test, y_pred))

In [ ]:
model.evaluate(np.array(X_test),np.array(y_test)) 

In [ ]:
from  sklearn.metrics import confusion_matrix 
confusion_matrix(y_test, y_pred) 

In [ ]:
## keras vgg16

In [ ]:
## Submission

In [ ]:
p,t = learn.get_preds(ds_type=DatasetType.Test)  

In [ ]:
p = np.array(p)
ids = np.array([f.name for f in (test_names+holdout_names)]) 
df_sub_fastai=pd.concat([pd.DataFrame(ids), pd.DataFrame(p[:, 1])], axis=1  )
df_sub_fastai.columns= df_sample.columns 

In [ ]:
df_sub_fastai[df_sub_fastai.has_oilpalm>0.5].shape,  df_sub_fastai[df_sub_fastai.has_oilpalm<=0.5].shape

In [ ]:
df_sub_fastai.to_csv(work_dir/'sub_fastai.csv', index=False)

In [ ]:
labels=np.argmax(p, 1)
df_sub_fastai2=pd.concat([pd.DataFrame(ids), pd.DataFrame(labels)], axis=1  )
df_sub_fastai2.columns= df_sample.columns 

In [ ]:
np.bincount(df_sub_fastai2.has_oilpalm) 

In [ ]:
os.listdir(work_dir) 

In [ ]:
from IPython.display import HTML
import base64
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_sub_fastai)

In [ ]:
create_download_link(df_sub_fastai2)

In [ ]:
df_sub_fastai.head(2)